In [1]:
import numpy as np
import pandas as pd

In [2]:
csv_base = './csv/'

csv_config = {
    'delimiter' : ';',
    'decimal' : ','
}

### Parámetros de entrada
Información que obtiene el modelo del sistema de información

In [3]:
# Demanda
demanda = 100
# Costo por unidad de tiempo
ctiempo = 1000

# Estos datos pueden ser obtenidos al consultar una vista en la base de datos
params_df = pd.read_csv(csv_base + 'entry_params.csv')
params_df.head()

,id_acopio,stock,precio,ppotencial,tiempo_alistam,ctransp,tiempo_transp
0,Santa Marta,30,100,20,2,3000,1
1,La Guajira,90,120,20,2,12000,4
2,Cordoba,10,105,40,2,6000,3


### Variables
Definición de variables para el modelo de optimización

In [4]:
N = params_df.shape[0]
# bounds = tuple((0, x_i) for x_i in params_df['stock'])

# Función objetivo
$$
\begin{align*}
    \sum_{i=0 \quad i\neq p}^{N} &\big[ \, kCA_i \times Precio(CA_i) + kCA_i \times cTransp(CA_i) + Tiempo(CA_i) \times cTiempo \, \big] \, + \\
                                 &\big[ \, kCA_p \times Precio(CA_p) + Demanda \times cTransp(CA_p) + Tiempo(CA_p) \times cTiempo \, \big]
\end{align*}
$$
Sujeto a las restricciones:
$$
\begin{align*}
    \sum_{i=0}^{N} kCA_i &= Demanda \\
    kCA_i &\leq Stock(CA_i) + Ppotencial(CA_i) &\therefore \, i=0,\cdots ,N \\
    TiempoAlistam(CA_i) &\leq TiempoMaxDefinido &\therefore \, i=0,\cdots ,N \\
    Tiempo(CA_i) &= TiempoAlistam(CA_i) + TiempoTransp(CA_i) \\
\end{align*}
$$

In [5]:
import random

def f(x):
    delta = 0
    # p = random.randint(0, N-1) 
    p = 0
    
    for i in range(N):
        if i == p or x[i] == 0:
            continue
        
        delta += get_delta(x, i)
    
    delta += get_delta(x, p, True)
    
    return delta

def get_delta(x, idx, p=False):
    kca = x[idx]
    precio = params_df['precio'].iloc[idx]
    ctransp = params_df['ctransp'].iloc[idx]
    tiempo = params_df['tiempo_transp'].iloc[idx] + params_df['tiempo_alistam'].iloc[idx]
    
    if not p:
        return (kca * precio) + (kca * ctransp) + (tiempo * ctiempo) 
    else:
        return (kca * precio) + (demanda * ctransp) + (tiempo * ctiempo)
    
        
f((50, 0, 10))

374050

## scipy

**LinearConstraint**: $ lb \leq A \cdot v[x] \leq ub $

**NonLinearConstraint**: $ lb \leq g(x) \leq ub $

In [6]:
# c = tuple(idx for (_, idx) in bounds)
# Aeq = np.ones((1, len(c)))
# beq = 1

In [7]:
# from scipy.optimize import linprog
# # from scipy.optimize import differential_evolution
# 
# result = linprog(
#     c,
#     A_ub=Aeq,
#     b_ub=beq,
#     bounds=bounds,
#     method='highs',
#     integrality=1
# )
# print(result)

## pymoo

In [8]:
from pymoo.core.problem import ElementwiseProblem

xl = np.zeros(N)
xu = np.asarray([x_i for x_i in params_df['stock']])

class Queso(ElementwiseProblem):
    def __init__(self):
        super().__init__(
            n_var=len(xl),
            n_obj=1,
            n_eq_constr=1,
            # n_ieq_constr=1,
            xl=xl,
            xu=xu,
            vtype=int
        )
        
    def _evaluate(self, x, out, *args, **kwargs):
        out['F'] = f(x)
        out['H'] = demanda - np.sum(x)

In [9]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.termination import get_termination

problem = Queso()

algorithm = GA(
    # pop_size=9,
    sampling=IntegerRandomSampling(),
    eliminate_duplicates=True
)

# termination = get_termination('time', '00:00:05')

res = minimize(problem,
               algorithm,
               # termination,
               seed=1,
               verbose=False)

print(f'F: {res.F}, \nX: {res.X}')


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False

F: [1201440.], 
X: [25 71  4]


In [10]:
from pymoo.core.sampling import Sampling
import random

class TopOrZeroSampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        vec = np.zeros(problem.n_var, dtype=np.int64)
        
        while np.sum(vec) < demanda:
            idx = np.random.randint(problem.n_var)
            vec[idx] += random.choice((0, xu[idx]))
            
            if np.sum(vec) > demanda:
                vec[idx] = vec[idx] - (np.sum(vec) - demanda)
                
        return [vec]

In [11]:
from pymoo.core.mutation import Mutation

class Mut(Mutation):
    def __init__(self):
        super().__init__()
        
    def _do(self, problem, X, **kwargs):
        
        for i in range(len(X)):
            r = np.random.random()
            
            if r < 0.4:
                idx = np.random.randint(len(X))
                
                if X[idx] == 0:
                    delta = xu[idx]
                    X[idx] = delta
                    ca = list(range(len(X))).remove(idx)
                    
                    while np.sum(X) > demanda:
                        temp = random.choice(ca)
                        if X[temp] + delta <= xu[temp]:
                            X[temp] = xu[temp]
                        else:
                            continue
                    
                else:
                    delta = xu[idx]
                    X[idx] = 0
                    ca = list(range(len(X))).remove(idx)
                    
                    while np.sum(X) < demanda:
                        temp = random.choice(ca)
                        if X[temp] + delta <= xu[temp]:
                            X[temp] = xu[temp]
                        else:
                            continue
                
            # elif r < 0.8:
                
        return X

In [12]:
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.repair.rounding import RoundingRepair

algorithm = GA(
    pop_size=20,
    sampling=TopOrZeroSampling(),
    crossover=SBX(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
    mutation=PM(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
    eliminate_duplicates=True
)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print(f'F: {res.F}, \nX: {res.X}')

F: [1141340.], 
X: [30 66  4]
